# 0.Import

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [ ]:
! pip install noun_splitter
! pip install torch transformers datasets
! pip install python-crfsuite

# Install transformers
%pip install -q transformers
! pip install --upgrade transformers

  Preparing metadata (setup.py) ... done
  Created wheel for noun_splitter: filename=noun_splitter-0.0.1-py3-none-any.whl size=11348 sha256=4eb917b17e0292d9c39efe19cab7ab2f20aab425821a74540980a70c7b52b700
  Stored in directory: /root/.cache/pip/wheels/28/65/4d/432d481215030fd34b9a79d9bec7f71ce7342171fa7cd312f0
Successfully built noun_splitter
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 39.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 93.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━

In [ ]:
! pip install python-crfsuite

In [ ]:
import json
import re
import torch
import time
import logging
import torch
import pandas as pd
import json
import warnings
warnings.filterwarnings("ignore")

from pathlib import Path
from transformers.data.processors.squad import SquadExample, squad_convert_examples_to_features
from transformers import AlbertForQuestionAnswering, TrainingArguments, Trainer
from transformers import AdamW, get_linear_schedule_with_warmup, AlbertTokenizer
from transformers import ElectraTokenizer, ElectraForQuestionAnswering, pipeline
from pprint import pprint
from transformers import TFAutoModel, AutoTokenizer, AutoModel
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from torch.utils.tensorboard import SummaryWriter
from sklearn.model_selection import train_test_split
from tqdm import trange, tqdm

In [ ]:
bar = '='
device = torch.device('cuda')
print(f'{bar*10}Device INFO{bar*10}')
print(f'Device :{device}')
print(bar*31)

==========Device INFO==========
Device :cuda


# 1.train data 준비

In [ ]:
def create_examples(input_data1,input_data2, set_type):
    is_training = set_type == "train"
    examples = []

    print("input_data1 처리 시작")
    filtered_data = [item for item in input_data1 if "금융" in item["doc_class"]["class"]]

    for item in filtered_data:
        title = item['doc_title']

        for para in item["paragraphs"]:
            context_text = para["context"]

            for qa in para["qas"]:
                qas_id = qa["question_id"]
                question_text = qa["question"]
                answer_text = qa["answer"]["text"]
                start_position_character = qa["answer"]["answer_start"]
                end_position_character = qa["answer"]["answer_end"]
                answers = []

                is_impossible = qa.get("is_impossible", False)

                # 정답 텍스트와 시작 위치를 가져와서 딕셔너리 형태로 만듦
                answer_dict = {
                    "text": answer_text,
                    "answer_start": start_position_character
                }

                # 만약 is_impossible가 False라면 정답을 answers 리스트에 추가
                if not is_impossible:
                    answers.append(answer_dict)

                example = SquadExample(
                    qas_id=qas_id,
                    question_text=question_text,
                    context_text=context_text,
                    answer_text=answer_text,
                    start_position_character=start_position_character,
                    title=title,
                    is_impossible=is_impossible,
                    answers=answers
                )

                examples.append(example)
    print("input_data1 처리 완료")

    print("input_data2 처리 시작")
    for index, row in input_data2.iterrows():
      title = row['title']
      context_text = row["context"]
      qas_id = row["question_id"]
      question_text = row["question"]
      answer_text = row["answer"]
      start_position_character = int(row["answer_start"])
      end_position_character = int(row["answer_end"])
      answers = []

      is_impossible = row.get("is_impossible", False)

      # 정답 텍스트와 시작 위치를 가져와서 딕셔너리 형태로 만듦
      answer_dict = {
          "text": answer_text,
          "answer_start": start_position_character
      }

      # 만약 is_impossible가 False라면 정답을 answers 리스트에 추가
      if not is_impossible:
          answers.append(answer_dict)

      example = SquadExample(
          qas_id=qas_id,
          question_text=question_text,
          context_text=context_text,
          answer_text=answer_text,
          start_position_character=start_position_character,
          title=title,
          is_impossible=is_impossible,
          answers=answers
      )

      examples.append(example)

    print("input_data2 처리 완료")


    train_examples, test_examples = train_test_split(examples, test_size=0.2, random_state=42)
    return train_examples, test_examples

In [ ]:
def get_train_examples(filename1, filename2):

  if filename1 :
    path = Path(filename1)

    with open(path, "rb") as f:
      input_data1 = json.load(f)["data"]

  if filename2 :
    input_data2 = pd.read_csv(filename2)

  return create_examples(input_data1,input_data2, "train")

In [ ]:
filename1 = "/content/gdrive/MyDrive/절미네 가족들/1.jiwon/1.data/0.row_data/AI_hub_fin_data.json"
filename2 = "/content/gdrive/MyDrive/절미네 가족들/1.jiwon/1.data/0.row_data/QA_set_final_ver3.csv"

train_examples, test_examples = get_train_examples(filename1,filename2)

input_data1 처리 시작
input_data1 처리 완료
input_data2 처리 시작
input_data2 처리 완료


In [ ]:
len(train_examples)

15876

In [ ]:
MODEL_PATH = "monologg/koelectra-base-v2-finetuned-korquad"

tokenizer = ElectraTokenizer.from_pretrained(MODEL_PATH)

model = ElectraForQuestionAnswering.from_pretrained(MODEL_PATH)

# 2.모델 training

In [ ]:
max_length = 512
doc_stride = 128
max_query_length = 64

train_features, train_dataset = squad_convert_examples_to_features(
    examples=train_examples,
    tokenizer=tokenizer,
    max_seq_length=max_length,
    doc_stride=doc_stride,
    max_query_length=max_query_length,
    is_training=True,
    return_dataset="pt",
)

# valid_features, valid_dataset = squad_convert_examples_to_features(
#     examples=train_examples,
#     tokenizer=tokenizer,
#     max_seq_length=max_length,
#     doc_stride=doc_stride,
#     max_query_length=max_query_length,
#     is_training=True,
#     return_dataset="pt",
# )

add example index and unique id: 100%|██████████| 15876/15876 [00:00<00:00, 601139.02it/s]


In [ ]:
logger = logging.getLogger(__name__)

train_batch_size = 16
gradient_accumulation_steps = 1
num_train_epochs = 10
max_grad_norm = 1.0

tb_writer = SummaryWriter()

train_sampler = RandomSampler(train_dataset)
train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=train_batch_size)
model = ElectraForQuestionAnswering.from_pretrained(MODEL_PATH)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

no_decay = ["bias", "LayerNorm.weight"]
optimizer_grouped_parameters = [
    {
        "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
        "weight_decay": 0.0,
    },
    {"params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0},
]
optimizer = AdamW(optimizer_grouped_parameters, lr=3e-5, eps=1e-8)
t_total = len(train_dataloader) //  gradient_accumulation_steps * num_train_epochs
scheduler = get_linear_schedule_with_warmup(
    optimizer, num_warmup_steps=0, num_training_steps=t_total
)

logger.info("***** Running training *****")
logger.info("  Num examples = %d", len(train_dataset))
logger.info("  Num Epochs = %d", num_train_epochs)
logger.info("  Gradient Accumulation steps = %d", gradient_accumulation_steps)
logger.info("  Total optimization steps = %d", t_total)

global_step = 1
epochs_trained = 0
steps_trained_in_current_epoch = 0
tr_loss, logging_loss = 0.0, 0.0
model.zero_grad()

for epoch in trange(epochs_trained, int(num_train_epochs), desc="Epoch", position=0, leave=True):
  epoch_iterator = tqdm(train_dataloader, desc=f"Epoch {epoch}", position=0, leave=True)
  for step, batch in enumerate(epoch_iterator):

    model.train()
    batch = tuple(t.to(device) for t in batch)

    inputs = {
      "input_ids": batch[0],
      "attention_mask": batch[1],
      "token_type_ids": batch[2],
      "start_positions": batch[3],
      "end_positions": batch[4],
    }

    outputs = model(**inputs)

    loss = outputs[0]
    loss.backward()

    tr_loss += loss.item()
    if (step + 1) % gradient_accumulation_steps == 0:
      torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)

      optimizer.step()
      scheduler.step()
      model.zero_grad()
      global_step += 1

    tb_writer.close()

Epoch: 100%|██████████| 10/10 [1:42:31<00:00, 615.15s/it]


In [ ]:
model.save_pretrained("/content/gdrive/MyDrive/절미네 가족들/1.jiwon/2.modeling/KOELECTRA_MODEL")

# 3. 모델 test

In [ ]:
MODEL_PATH = "monologg/koelectra-base-v2-finetuned-korquad"
MODEL_PATH_2 = "/content/gdrive/MyDrive/절미네 가족들/1.jiwon/2.modeling/KOELECTRA_MODEL/pytorch_model.bin"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = ElectraForQuestionAnswering.from_pretrained(MODEL_PATH)
model.load_state_dict(torch.load(MODEL_PATH_2))
model.to(device)

# 텍스트 Tokenizer
tokenizer = ElectraTokenizer.from_pretrained(MODEL_PATH)

In [ ]:
test_examples_1 = test_examples[0:100]

In [ ]:
max_length = 512
doc_stride = 128
max_query_length = 64

test_features, test_dataset = squad_convert_examples_to_features(
    examples=test_examples_1,
    tokenizer=tokenizer,
    max_seq_length=max_length,
    doc_stride=doc_stride,
    max_query_length=max_query_length,
    is_training=True,
    return_dataset="pt",
)

train_batch_size = 16

test_sampler = RandomSampler(test_dataset)
test_dataloader = DataLoader(test_dataset, sampler=test_sampler, batch_size=train_batch_size)

add example index and unique id: 100%|██████████| 100/100 [00:00<00:00, 259067.57it/s]


In [ ]:
# 모델을 평가 모드로 설정합니다.
model.eval()

# 예측 및 실제 답변을 저장할 리스트를 준비합니다.
predictions = []
actuals = []

# 데이터로더에서 테스트 데이터셋을 순회합니다.
for batch in test_dataloader:
    # 입력을 GPU로 전송합니다.
    batch = tuple(t.to(device) for t in batch)

    with torch.no_grad():
        inputs = {
            "input_ids": batch[0],
            "attention_mask": batch[1],
            "token_type_ids": batch[2]
        }
        # 모델을 실행하여 예측 시작 및 종료 위치를 얻습니다.
        outputs = model(**inputs)
        start_logits, end_logits = outputs[:2]

    # 예측 시작 및 종료 위치의 인덱스를 얻습니다.
    start_indices = torch.argmax(start_logits, dim=1)
    end_indices = torch.argmax(end_logits, dim=1)

    # 예측된 텍스트 답변을 추출합니다.
    for i in range(len(batch[0])):
        input_ids = batch[0][i].tolist()
        pred_answer = tokenizer.decode(input_ids[start_indices[i]:end_indices[i]+1])
        actual_answer = tokenizer.decode(input_ids[batch[3][i]:batch[4][i]+1])
        predictions.append(pred_answer)
        actuals.append(actual_answer)

In [ ]:
import re
import string
import collections

def normalize_answer(s):
    """Lower text and remove punctuation, articles, and extra whitespace."""
    def remove_articles(text):
        return re.sub(r'\b(a|an|the)\b', ' ', text)

    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))

def compute_exact(a_gold, a_pred):
    return int(normalize_answer(a_gold) == normalize_answer(a_pred))

def compute_f1(a_gold, a_pred):
    gold_toks = normalize_answer(a_gold).split()
    pred_toks = normalize_answer(a_pred).split()
    common = collections.Counter(gold_toks) & collections.Counter(pred_toks)
    num_same = sum(common.values())
    # if len(gold_toks) == 0 or len(pred_toks) == 0:
    #     return int(gold_toks == pred_toks)
    if num_same == 0:
        return 0
    precision = 1.0 * num_same / len(pred_toks)
    recall = 1.0 * num_same / len(gold_toks)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1

# EM 및 F1 점수를 계산합니다.
f1_total = 0
exact_total = 0

for prediction, actual in zip(predictions, actuals):
    f1_total += compute_f1(actual, prediction)
    exact_total += compute_exact(actual, prediction)

f1 = f1_total / len(predictions)
exact_match = exact_total / len(predictions)

print("F1 Score:", f1)
print("Exact Match (EM) Score:", exact_match)

F1 Score: 0.9075418391955734
Exact Match (EM) Score: 0.8442622950819673


# 4.모델 시연

In [ ]:
test_data = pd.read_csv('/content/gdrive/MyDrive/절미네 가족들/1.jiwon/1.data/0.row_data/QA_set_final_ver3.csv')

In [ ]:
key = test_data['question']
value = test_data['context']

In [ ]:
from pprint import pprint
from transformers import pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

def find_most_similar_context(question, question2, context):
    vectorizer = TfidfVectorizer().fit(question2)
    tfidf_matrix = vectorizer.transform(question2)
    question_vector = vectorizer.transform([question])

    # 질문의 TF-IDF 벡터와 모든 질문의 TF-IDF 벡터 사이의 코사인 유사도를 계산
    cosine_similarities = linear_kernel(question_vector, tfidf_matrix).flatten()

    # 가장 유사한 컨텍스트의 인덱스를 찾음
    best_context_index = cosine_similarities.argmax()

    return context[best_context_index]

qa = pipeline("question-answering", tokenizer=tokenizer, model=model, device=0)

In [ ]:
question = "채권이란?"
best_context = find_most_similar_context(question, key, value)

In [ ]:
best_context

'채권은 정부, 공공기관, 특수법인 등과 주식회사의 형태를 갖춘 기업이 비교적 거액의 장기자금을 일시에 대량으로 조달하기 위하여 발행하는 일종의 차용증서이다. 채권은 다음과 같은 몇 가지 특징을 갖고 있다. 채권은 확정이자부 증권(fixed income securities)으로서 발행 시에 채무자가 지급해야 하는 이자와 상환금액이 확정되어 있거나 또는 그 기준이 확정되어 있는 것이 특징이다. 이때 발행된 채권 1장의 가치를 액면가(par value)라고 하고 이것이 만기 시점에 채권 보유자에게 상환해야 할 금액이 된다. 채권은 주식의 경우와는 달리 수익의 발생여부와 관계없이 이자를 지급해야 한다. 즉, 주식의 경우 순이익 발생여부에 따라 배당정책이 다르게 될 수 있으나, 채권은 적자가 발생하더라도 이미 확정되어 있는 이자를 지급해야 한다는 점에서 주식과 차이가 있다. 이때 액면에 대해 약정한 이자율을 연율로 표시한 것을 표면금리(coupon rate)라고 한다.'

In [ ]:
question

'채권이란?'

In [ ]:
# 가장 유사한 컨텍스트로 답변 얻기

answer = qa({"question": question, "context": best_context})
print(answer)

{'score': 1.9956388541686465e-07, 'start': 80, 'end': 87, 'answer': '차용증서이다.'}




---



In [ ]:
# 한번 따로 넣어서 해보기

context = """
기업들은 생산요소를 구입하여 생산 활동을 함으로써 매출과 이익을 올리고 있다.
성공적인 경제활동을 했다면 비용보다 수입이 크게 될 것이다. 기업의 소득은 주주들에게 배당하기도 하고 새로운 사업을 위하여 직접 보유하기도 한다.
그럼에도 기업은 현재의 생산을 유지하거나 새로운 사업을 위해서 항상 자금이 필요하다.
기업은 생산물 판매가 활발하게 일어나면 시설을 확장하기 위해서 돈이 필요하고, 반대로 판매 수입이 줄어들면 부족한 돈을 빌리게 되므로 통상적으로 자금의 수요자가 된다.
가계·기업·정부 모두 수입보다 지출이 많으면 적자가 되고 반대로 수입보다 지출이 적으면 흑자가 된다.
흑자 주체는 자신들의 돈을 효과적으로 활용하기 위하여 다양한 방법으로 돈을투자하게 되고 반대로 적자 주체는 필요한 돈을 마련하기 위해서 돈을 빌리게 된다.
이와 같이 흑자 주체에서 적자 주체로 돈이 이동하는 것을 금융이라 한다.
즉 여유자금이 있는 흑자 주체(저축자: 공급자)가 돈을 빌려주거나 적자 주체(차입자: 수요자)가 돈을 빌리는 것을 금융이라 하고 금융거래가 일어나는 곳을 금융시장이라고 한다.
생산·소비·유통·분배와 같은 경제활동이 원활하게 일어나기 위해서는 각 경제주체 간에 일어나는 거래를 뒷받침할 수 있는 돈의 양이 충분해야 하고
돈의 흐름을 원활하게 해주는 금융시장이 잘 발달되어 있어야 한다.선진국들은 생산물의 크기도 크지만 경제활동을
뒷받침하는 금융시장도 잘 발달되어 있는 데 비해서 개발도상국들은 상대적으로 생산성도 낮고 금융시장도 낙후되어 있다.
"""

context2 =  """
즉 여유자금이 있는 흑자 주체(저축자: 공급자)가 돈을 빌려주거나 적자 주체(차입자: 수요자)가 돈을 빌리는 것을 금융이라 하고 금융거래가 일어나는 곳을 금융시장이라고 한다.
"""

answer = qa({"question": '경제활동이 원활하게 일어나기 위해 필요한 것은 무엇인가요?', "context": context})
print(answer)